In [1]:
import numpy as np
from sklearn import datasets
import matplotlib.pyplot as plt
import itertools
%matplotlib inline

## task 01

* Измените функцию calc_logloss так, чтобы нули по возможности не попадали в np.log.
* Подберите аргументы функции eval_model для логистической регрессии таким образом, чтобы log loss был минимальным.
* Создайте функцию calc_pred_proba, возвращающую предсказанную вероятность класса 1 (на вход подаются W, который уже посчитан функцией eval_model и X, на выходе - массив y_pred_proba).
* Создайте функцию calc_pred, возвращающую предсказанный класс (на вход подаются W, который уже посчитан функцией eval_model и X, на выходе - массив y_pred).
* Посчитайте Accuracy, матрицу ошибок, точность и полноту, а также F1 score.
* Могла ли модель переобучиться? Почему?
* Создайте функции eval_model_l1 и eval_model_l2 с применением L1 и L2 регуляризаций соответственно.

In [2]:
def calc_logloss(y, y_pred):
    eps = 1e-10
    y_pred[y_pred == 0] = 0 + eps
    y_pred[y_pred == 1] = 1 - eps
    err = - np.mean(y * np.log(y_pred) + (1.0 - y) * np.log(1.0 - y_pred))
    return err

In [3]:
# y1 = np.array([1, 0])
# y_pred1 = np.array([0.001, 1])
# calc_logloss(y1, y_pred1)

In [4]:
def sigmoid(z):
    return 1 / (1 + np.exp(-z))

In [5]:
def eval_model(X, y, max_iter, alpha):
    W = np.zeros(2)
    n = X.shape[1]
    eps = 1e-8
    weight_dist = np.inf
    i = 0
    while weight_dist > eps and i < max_iter:
        
        z = np.dot(W, X)
        y_pred = sigmoid(z)
        err = calc_logloss(y, y_pred)
        W_new = W - alpha * (1/n * np.dot((y_pred - y), X.T))
        
        weight_dist = np.linalg.norm(W_new - W, ord = 2)
        i += 1
        W = W_new
    return W, i, err

In [6]:
def best_params(X, y, *params):
    a = []
    for vals in itertools.product(iters, etas):
        W, i, err = eval_model(X, y, *vals) # для логистической регрессии eval_model!
        a.append([*vals, W, i, err])

    ind = [params[-1] for params in a].index(min([params[-1] for params in a]))
    best_err = a[ind][4]
    best_eta = a[ind][1]
    if a[ind][0] > a[ind][3]: #алгоритм сошелся раньше, чем достигли max_iter
        best_iter = a[ind][3]
    else: #алгоритм так и не сошелся при данных eta, max_iters
        best_iter = a[ind][0] 
        print('алгоритм не сошелся')

    return best_iter, best_err, best_eta

In [7]:
def calc_pred_proba(W, X):
    y_pred_proba = sigmoid(W.T.dot(X))
    return y_pred_proba

In [8]:
def calc_pred(W, X, threshold = 0.5):

    y_pred = calc_pred_proba(W, X)
    y_pred[y_pred >= threshold] = 1
    y_pred[y_pred < threshold] = 0
    return y_pred 

In [9]:
def confusion_matrix(y, y_pred):
    TP = ((y_pred == 1) & (y == 1)).sum()
    FP = ((y_pred == 1) & (y == 0)).sum()
    FN = ((y_pred == 0) & (y == 1)).sum()
    TN = ((y_pred == 0) & (y == 0)).sum()
    
    return np.array([[TP, FP], [FN, TN]])

In [10]:
def classification_report(y, y_pred):
    TP = ((y_pred == 1) & (y == 1)).sum()
    FP = ((y_pred == 1) & (y == 0)).sum()
    FN = ((y_pred == 0) & (y == 1)).sum()
    TN = ((y_pred == 1) & (y == 0)).sum()
    
    precision = TP / (TP + FP)
    recall = TP / (TP + FN)
    F1_score = 2 * precision * recall / (precision + recall)
    
    return precision, recall, F1_score

In [11]:
# генерация данных
# разница между метриками начинает появляться только после увеличения параметра генерации flip_y до 0.10, 
# на менее шумных данных нет разницы между трейном и тестом, регуляризация тоже ничего не меняет
classes = datasets.make_classification(n_samples=100, 
                                       n_features=2, 
                                       n_informative=2,
                                       n_redundant=0, 
                                       n_classes=2, 
                                       flip_y = 0.10, #шум
                                       weights = [0.833, 0.167], #соотношение классов
                                       random_state=1)
X, y = classes[0].T, classes[1]

In [12]:
# наборы параметров
etas = np.array([1e-2, 1e-4, 1e-6])
iters = np.array([1000, 10000, 100000])

### обучаем модель на всем датасете

In [13]:
best_iter, best_err, best_eta = best_params(X, y, etas, iters)
best_iter, best_err, best_eta

(36742, 0.17105028128073477, 0.01)

In [14]:
W, i, err = eval_model(X, y, best_iter, best_eta)
W, err

(array([-0.11943017,  3.36114494]), 0.17105028128073477)

In [15]:
y_pred_probs = calc_pred_proba(W, X)

In [16]:
y_pred = calc_pred(W, X)

In [17]:
# list(zip(y_pred_probs, y_pred, y))[:10]

### считаем метрики на всем датасете

In [18]:
accuracy = np.mean(y_pred == y)
accuracy

0.95

In [19]:
c_matrix = confusion_matrix(y, y_pred)
c_matrix

array([[15,  1],
       [ 4, 80]])

In [20]:
precision, recall, F1_score = classification_report(y, y_pred)
precision, recall, F1_score

(0.9375, 0.7894736842105263, 0.8571428571428572)

### делим выборку на обучающую и тестовую

In [21]:
# перемешивание датасета
np.random.seed(12)
shuffle_index = np.random.permutation(classes[0].shape[0])
X_shuffled, y_shuffled = classes[0][shuffle_index], classes[1][shuffle_index]

# разбивка на обучающую и тестовую выборки
train_proportion = 0.7
train_test_cut = int(len(classes[0]) * train_proportion)

X_train, X_test, y_train, y_test = \
    X_shuffled[:train_test_cut], \
    X_shuffled[train_test_cut:], \
    y_shuffled[:train_test_cut], \
    y_shuffled[train_test_cut:]
    
X_train = X_train.T
X_test = X_test.T

### обучаем на X_train

In [22]:
best_iter_train, best_err_train, best_eta_train = best_params(X_train, y_train, etas, iters)
best_iter_train, best_err_train, best_eta_train

алгоритм не сошелся


(100000, 0.0716071421399939, 0.01)

In [23]:
W_, i_, err_ = eval_model(X_train, y_train, best_iter_train, best_eta_train)
W_, i_, err_

(array([0.36426709, 5.00824682]), 100000, 0.0716071421399939)

In [24]:
y_pred_probs_train = calc_pred_proba(W_, X_train)
y_pred_train = calc_pred(W_, X_train)

In [25]:
confusion_matrix(y_train, y_pred_train)

array([[10,  0],
       [ 1, 59]])

In [26]:
classification_report(y_train, y_pred_train)

(1.0, 0.9090909090909091, 0.9523809523809523)

### проверяем на X_test

In [27]:
y_pred_probs_test = calc_pred_proba(W_, X_test)
y_pred_test = calc_pred(W_, X_test)

In [28]:
confusion_matrix(y_test, y_pred_test)

array([[ 6,  1],
       [ 2, 21]])

In [29]:
classification_report(y_test, y_pred_test)

(0.8571428571428571, 0.75, 0.7999999999999999)

При таком уровне шума на тесте результаты уже хуже, чем на трейне

### L1, L2 регуляризация

In [30]:
def eval_model_l1(X, y, max_iter, alpha, lambda_ = 1e-8):
    W = np.zeros(2)
    n = X.shape[1]
    eps = 1e-8
    weight_dist = np.inf
    i = 0
    while weight_dist > eps and i < max_iter:
        z = np.dot(W, X)
        y_pred = sigmoid(z)
        err = calc_logloss(y, y_pred)
        W_new = W - alpha * (1/n * np.dot((y_pred - y), X.T) + lambda_* np.sign(W))
        
        weight_dist = np.linalg.norm(W_new - W, ord = 2)
        i += 1
        W = W_new
    return W, i, err

In [31]:
def eval_model_l2(X, y, max_iter, alpha, lambda_ = 1e-8):
    W = np.zeros(2)
    n = X.shape[1]
    eps = 1e-8
    weight_dist = np.inf
    i = 0
    while weight_dist > eps and i < max_iter:
        z = np.dot(W, X)
        y_pred = sigmoid(z)
        err = calc_logloss(y, y_pred)
        W_new = W - alpha * (1/n * np.dot((y_pred - y), X.T) + 2 * lambda_* W)
        
        weight_dist = np.linalg.norm(W_new - W, ord = 2)
        i += 1
        W = W_new
    return W, i, err

In [32]:
lambdas = np.array([1e-4,1e-6, 1e-8, 1e-10])

In [33]:
l1 = []
for l in lambdas:
    W, i, err = eval_model_l1(X_train, y_train, best_iter_train, best_eta_train, l)
    l1.append([l, W, i, err])

ind = [params[-1] for params in l1].index(min([params[-1] for params in l1]))
best_lambda_train = l1[ind][0]
best_err_train = l1[ind][-1]
best_i = l1[ind][2]
best_lambda_train, best_err_train, best_i

(1e-10, 0.07160714214026109, 100000)

In [34]:
l2 = []
for l in lambdas:
    W, i, err = eval_model_l2(X_train, y_train, best_iter_train, best_eta_train, l)
    l2.append([l, W, i, err])

ind = [params[-1] for params in l2].index(min([params[-1] for params in l2]))
best_err_train = l2[ind][-1]
best_i = l2[ind][2]
best_lambda_train, best_err_train, best_i

(1e-10, 0.07160714214234898, 100000)

разницы между моделями без регуляризации, L1 b L2 в данном случае практически никакой

### L2 X_train vs X_test

In [35]:
W_l2, i_l2, err_l2 = eval_model_l2(X_train, y_train, best_iter_train, best_eta_train, best_lambda_train)
W_l2, err_l2

(array([0.36426706, 5.00824666]), 0.07160714214234898)

In [36]:
y_pred_probs_train_l2 = calc_pred_proba(W_l2, X_train)
y_pred_train_l2 = calc_pred(W_l2, X_train)

In [37]:
confusion_matrix(y_train, y_pred_train_l2)

array([[10,  0],
       [ 1, 59]])

In [38]:
classification_report(y_train, y_pred_train_l2)

(1.0, 0.9090909090909091, 0.9523809523809523)

In [39]:
y_pred_probs_test_l2 = calc_pred_proba(W_l2, X_test)
y_pred_test_l2 = calc_pred(W_l2, X_test)

In [40]:
confusion_matrix(y_test, y_pred_test_l2)

array([[ 6,  1],
       [ 2, 21]])

In [41]:
classification_report(y_test, y_pred_test_l2)

(0.8571428571428571, 0.75, 0.7999999999999999)

Получили практически те же результаты, что и на модели без регуляризации, за исключением ошибки - различается в 10м знаке после запятой (best_lambda = 1e-10, а веса получаются небольшими)
